In [ ]:
import pandas as pd
import numpy as np

In [ ]:
def read_excel(path="./data.xlsx"):
    data_xls = pd.io.excel.ExcelFile(path)
    data={}
    print(data_xls.sheet_names)
    for name in data_xls.sheet_names:
        df=pd.read_excel(data_xls,sheet_name=name)
        data[name]=df
        # print(name, " : df = ",df , type(df))
        if df.empty:
            continue
    return data

data = read_excel()
data

In [ ]:
pinformation = data["驾驶员基本信息"]
pwork = data["工作时长"]
service_violation = data["服务违章"]
safety_violation = data["安全违章"]
safety_accident = data["安全事故"]

In [ ]:
# pinformation
# 男 - 1 ； 女 - 0
pinformation["性别"].value_counts()
pinformation.loc[pinformation["性别"]=='男' , '性别']=1
pinformation.loc[pinformation["性别"]=='女' , '性别']=0
pinformation

In [ ]:
pinformation["分公司"].value_counts()
pinformation = pinformation.drop(labels="分公司", axis=1)
pinformation

In [ ]:
pinformation["学历"].value_counts()
# 普高 0，中专 1， 初中 2， 大专 3， 高中（普高） 4， 本科 5， 技校 6， 职高 7 ，小学 8， nan 9
pinformation.loc[pinformation["学历"]=='普高' , '学历']=0
pinformation.loc[pinformation["学历"]=='中专' , '学历']=1
pinformation.loc[pinformation["学历"]=='初中' , '学历']=2
pinformation.loc[pinformation["学历"]=='大专' , '学历']=3
pinformation.loc[pinformation["学历"]=='高中（普高）' , '学历']=4
pinformation.loc[pinformation["学历"]=='本科' , '学历']=5
pinformation.loc[pinformation["学历"]=='技校' , '学历']=6
pinformation.loc[pinformation["学历"]=='职高' , '学历']=7
pinformation.loc[pinformation["学历"]=='小学' , '学历']=8
pinformation["学历"] = pinformation["学历"].fillna(9)
pinformation

In [ ]:
pinformation['出生日期'] = pd.to_datetime(pinformation['出生日期'])
import datetime as dt
now_year = dt.datetime.today().year
# pinformation['age'] = now_year - frame.birth.dt.year
pinformation['出生日期'] = now_year - pinformation['出生日期'].dt.year

pinformation['入职日期'] = pd.to_datetime(pinformation['入职日期'])
now_year = dt.datetime.today().year
# pinformation['age'] = now_year - frame.birth.dt.year
pinformation['入职日期'] = now_year - pinformation['入职日期'].dt.year

pinformation

In [ ]:
# pinformation = pinformation.drop(labels=["事故", "安全违章", "服务违章"], axis=1)


In [ ]:
safety_accident.columns = safety_accident.columns.str.replace('事故日期', '开班日期')

safety_accident['开班日期'] = pd.to_datetime(safety_accident['开班日期'],format='%Y-%M-%d').dt.date.astype(object)
safety_accident
pwork['开班日期'] = pd.to_datetime(pwork['开班日期'],format='%Y-%M-%d').dt.date.astype(object)
res = pd.merge(pwork, safety_accident, how='right', on=['员工编号', '开班日期'])
res
# print(safety_accident)

In [ ]:
import matplotlib.pyplot as plt
y_1 = res['工作时长（小时）']
y_2 = res['驾车时长（小时）']
y_3 = res['运营时长（小时）']
x = [i for i in range(len(y_1))]
plt.plot(x, y_1)
plt.plot(x, y_2)
plt.plot(x, y_3)
plt.show()

print(res)
import seaborn as sns
hres = res[["工作时长（小时）", "驾车时长（小时）", "运营时长（小时）"]]
plot=sns.heatmap(res)
plt.show()

In [ ]:
hh = pd.merge(pwork, safety_accident, how='left', on=['员工编号'])
hh.insert(loc=len(hh.columns.array), column="事故", value = 1)
hh.loc[hh['事故原因'].isna(), "事故"]  = 0
print(hh)
import seaborn as sns
hres = hh[["工作时长（小时）", "驾车时长（小时）", "运营时长（小时）", "事故"]]

plt.subplots(figsize=(20,15))
ax = plt.axes()
ax.set_title("Correlation Heatmap")
corr = hres.corr()
sns.heatmap(corr,
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values)
plt.show()

In [ ]:
oth_res = pd.merge(pwork, safety_accident, how='left', on=['员工编号'])
print(oth_res)
oth_y_1 = oth_res[oth_res['开班日期_y'].isna()]['工作时长（小时）']
oth_y_2 = oth_res[oth_res['开班日期_y'].isna()]['驾车时长（小时）']
oth_y_3 = oth_res[oth_res['开班日期_y'].isna()]['运营时长（小时）']
oth_x = [i for i in range(len(oth_y_1))]
plt.subplot(3, 2, 1)
plt.scatter(x, y_1)
plt.subplot(3, 2, 2)
plt.scatter(oth_x, oth_y_1)
plt.show()
plt.subplot(3, 2, 3)
plt.scatter(x, y_2)
plt.subplot(3, 2, 4)
plt.scatter(oth_x, oth_y_2)
plt.show()
plt.subplot(3, 2, 5)
plt.scatter(x, y_3)
plt.subplot(3, 2, 6)
plt.scatter(oth_x, oth_y_3)
plt.show()

In [ ]:
print(service_violation.groupby(service_violation['服务违章项目']).value_counts())

In [ ]:
service_res = pd.merge(service_violation, safety_accident, how='left', on=['员工编号'])
print(service_res)
no_num = len(service_res[oth_res['开班日期_y'].isna()])
ys_num = len(service_res['服务违章项目']) - no_num
plt.bar([0, 1], [no_num, ys_num],  width=0.5)
plt.show()
plt.pie([float(no_num)/float(no_num + ys_num), float(ys_num)/float(no_num + ys_num)])
plt.show()

In [ ]:
# pinformation
# pinformation = pinformation.drop(labels=[ "服务违章"], axis=1)
pinformation.insert(loc=6, column="服务违章", value = 0)

pinformation.loc[pinformation["员工编号"].isin(service_violation["员工编号"]), "服务违章"] = 1
pinformation

In [ ]:
safety_res = pd.merge(safety_violation, safety_accident, how='left', on=['员工编号'])
print(safety_res)
sno_num = len(safety_res[safety_res['开班日期'].isna()])
sys_num = len(safety_res['安全违章项目']) - sno_num
plt.bar([0, 1], [sno_num, sys_num],  width=0.5)
plt.show()
plt.pie([float(sno_num)/float(sno_num + sys_num), float(sys_num)/float(sno_num + sys_num)])
plt.show()

In [ ]:

pinformation.insert(loc=7, column="安全违章", value = 0)
pinformation
pinformation.loc[pinformation["员工编号"].isin(safety_violation["员工编号"]), "安全违章"] = 1
print(pinformation["安全违章"].value_counts())

In [ ]:
pinformation.insert(loc=8, column="事故", value = 0)
pinformation
pinformation.loc[pinformation["员工编号"].isin(safety_accident["员工编号"]), "事故"] = 1
pinformation

In [ ]:
pinformation = pinformation.drop(labels=["初领证日期"], axis=1)
pinformation = pinformation.drop(labels=["员工编号"], axis=1)
# pinformation = pinformation.drop(labels=["入职日期"], axis=1)
pinformation
print(pinformation['事故'].value_counts())
pinformation.to_csv("./data1.csv", index = False)

In [ ]:
print(pinformation[pinformation['安全违章']==1]['事故'].value_counts())

In [ ]:
data_list = pinformation.values.tolist()
print(data_list)
label_list = []
for i in range(len(data_list)):
    label_list.append([data_list[i][-1]])
    data_list[i] = data_list[i][:-1]
print(data_list)
print(label_list)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report


def get_data():
    """
    获取数据集
    :return:
    """
    iris = load_iris()

    return iris.data, iris.target


def split_data(data, target):
    """
    划分数据集
    :param data:
    :param target:
    :return:
    """
    data = np.array(data)
    target = np.array(target)
    target = np.reshape(target, (target.shape[0], 1))

    # 正则化数据，防止数据大小本身对结果造成影响
    sd = StandardScaler()
    data = sd.fit_transform(data)

    # 拼接特征值与类别
    dataset = np.hstack((data, target))
    n = dataset.shape[0]

    # 打乱数据
    np.random.shuffle(dataset)

    # 划分数据集，返回训练集与测试集
    train = dataset[:int(0.75 * n), :]
    test = dataset[int(0.75 * n):, :]

    return train, test


def sigmoid(z):
    """
    sigmoid函数
    :param z:
    :return:
    """
    return 1 / (1 + np.exp(-z))


def draw_sigmoid():
    """
    画出sigmoid函数
    :return:
    """
    fig, ax = plt.subplots()
    x_data = np.arange(-10, 10, 0.1)
    ax.plot(x_data, sigmoid(x_data))
    plt.show()


def calCost(dataset, theta):
    """
    计算代价函数
    :param dataset:
    :param theta:
    :return:
    """
    x = dataset[:, :-1]
    y = dataset[:, -1:]
    z = x @ theta.T
    # 训练数据个数,或者用m = y.shape[1]
    m = y.size
    para1 = np.multiply(-y, np.log(sigmoid(z)))
    para2 = np.multiply((1 - y), np.log(1 - sigmoid(z)))
    # 代价函数Y
    J = 1 / m * np.sum(para1 - para2)
    return J


def gradient(dataset, theta, iters, alpha):
    """
    梯度下降
    :param dataset:
    :param theta:
    :param iters:
    :param alpha:
    :return:
    """
    # 存放每次梯度下降后的损失值
    x = dataset[:, :-1]
    y = dataset[:, -1:]
    for i in range(iters):
        print("T: ", theta.T)
        h_x = sigmoid(x @ theta.T)

        theta = theta - alpha / len(x) * (h_x - y).T @ x
    return theta


def get_per_classify_data(data, i):
    """
    返回第i类的数据
    :param data:数据集
    :param i:类别
    :return:
    """
    return data[data[:, -1] == i]


def get_final_theta(data, i, theta, iters, alpha):
    """
    获取梯度下降后的theta值
    :param data:
    :param i:
    :param theta:
    :param iters:
    :param alpha:
    :return:
    """
    dataset = get_per_classify_data(data, i)
    return gradient(dataset, theta, iters, alpha)


def predict(dataset, theta_list):
    """
    预测结果
    :param dataset:
    :param theta_list:
    :return:
    """
    x = dataset[:, :-1]
    per_theta_list = [i[0] for i in theta_list]
    per_theta_list = np.array(per_theta_list)

    per_prob = sigmoid(np.dot(x, per_theta_list.T))

    # 返回每行最大值所在的索引，即概率最大的类别
    # print(np.max(per_prob, axis=1))
    # return np.argmax(per_prob, axis=1)
    return per_prob

from scipy.interpolate import make_interp_spline

from sklearn.metrics import precision_recall_curve, roc_curve
from sklearn.linear_model import LogisticRegression

if __name__ == '__main__':
    plt.rcParams['font.sans-serif'] = 'SimHei'  # 黑体
    plt.rcParams['axes.unicode_minus'] = False  # 显示负号

    data, target = get_data()

    target_list = []
    for i in label_list:
        target_list.append(i[0])

    # print(data.shape, target.shape, np.array(data_list).shape, np.array(target_list).shape)
    # print(target_list)
    train, test = split_data(np.array(data_list), np.array(target_list))
    # draw_sigmoid()

    iters = 1000 # 迭代次数
    alpha = 0.5 # 学习率
    theta_list = []
    for i in range(np.array(data_list).shape[1]):
        theta = np.zeros((1, np.array(data_list).shape[1]))
        theta_list.append(theta)

    final_theta_list = []
    target_list1 = list(set(np.array(target_list)))

    for i in target_list1:
        theta_i = get_final_theta(train, i, theta_list[target_list1.index(i)], iters, alpha)
        final_theta_list.append(theta_i)


    y_predict = predict(test, final_theta_list)
    res_pre = []
    # print(y_predict.shape,test.shape)
    for i in range(len(test)):
        # print(test[i][-1], y_predict[i])
        res_pre.append(y_predict[i][int(test[i][-1])])
        # print(test[i][-1], y_predict[i], res_pre[i])
    Precision,Recall,throds = precision_recall_curve(test[:, -1], res_pre)
    FPR,TPR,_ = roc_curve(test[:, -1], res_pre)
    # print(y_predict.shape, test[:, -1].shape, Precision,Recall, throds)

    plt.xlabel("Recall")
    plt.ylabel("Preccision")
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])

    plt.plot(Precision, Recall)
    plt.show()
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.plot(FPR, TPR)
    plt.show()
    # 查看预测准确度
    # print(classification_report(y_predict, test[:, -1]))
